# Metrics of matching (simple)
Example of the functions to plot match_metrics of matching.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pylab as plt

## Generate random data and add to catalog

In [ ]:
# For reproducibility
np.random.seed(1)

In [ ]:
from support import gen_cluster
input1, input2 = gen_cluster()

In [ ]:
from clevar import ClCatalog
c1 = ClCatalog('Cat1', ra=input1['RA'], dec=input1['DEC'], z=input1['Z'], mass=input1['MASS'],
            mass_err=input1['MASS_ERR'], z_err=input1['Z_ERR'])
c2 = ClCatalog('Cat2', ra=input2['RA'], dec=input2['DEC'], z=input2['Z'], mass=input2['MASS'],
            mass_err=input2['MASS_ERR'], z_err=input2['Z_ERR'])
# Format for nice display
for c in ('ra', 'dec', 'z', 'z_err'):
    c1[c].info.format = '.2f'
    c2[c].info.format = '.2f'
for c in ('mass', 'mass_err'):
    c1[c].info.format = '.2e'
    c2[c].info.format = '.2e'

## Match catalogs

In [ ]:
from clevar.match import ProximityMatch
from clevar.cosmology import AstroPyCosmology

match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'which_radius': 'max', # Case of radius to be used, can be: cat1, cat2, min, max
    'preference': 'angular_proximity', # options are more_massive, angular_proximity or redshift_proximity
    'catalog1': {'delta_z':.2,
                'match_radius': '1 mpc'
                },
    'catalog2': {'delta_z':.2,
                'match_radius': '10 arcsec'
                }
}

cosmo = AstroPyCosmology()
mt = ProximityMatch()
mt.match_from_config(c1, c2, match_config, cosmo=cosmo)

## Recovery rate
Compute recovery rates, they are computed in mass and redshift bins. There are several ways they can be displayed:
- Single panel with multiple lines
- Multiple panels
- 2D color map

In [ ]:
from clevar.match_metrics import recovery

### Simple plot
The recovery rates are shown as a function of redshift in mass bins.
They can be displayed as a continuous line or with steps:

In [ ]:
zbins = np.linspace(0, 2, 21)
mbins = np.logspace(13, 14, 5)
info = recovery.plot(c1, 'cross', zbins, mbins)
plt.show()
info = recovery.plot(c1, 'cross', zbins, mbins, shape='line')
plt.show()

They can also be transposed to be shown as a function of mass in redshift bins.

In [ ]:
zbins = np.linspace(0, 2, 5)
mbins = np.logspace(13, 14, 20)
info = recovery.plot(c1, 'cross', zbins, mbins,
                     shape='line', transpose=True)

The full information of the recovery rate histogram in a dictionay containing:

* `data`: Binned data used in the plot. It has the sections:

    * `recovery`: Recovery rate binned with (bin1, bin2).\
    bins where no cluster was found have nan value.
    * `edges1`: The bin edges along the first dimension.
    * `edges2`: The bin edges along the second dimension.
    * `counts`: Counts of all clusters in bins.
    * `matched`: Counts of matched clusters in bins.


In [ ]:
info['data'].keys()

### Panels plots
You can also have a panel for each bin:

In [ ]:
zbins = np.linspace(0, 2, 21)
mbins = np.logspace(13, 14, 5)
info = recovery.plot_panel(c1, 'cross', zbins, mbins)

zbins = np.linspace(0, 2, 5)
mbins = np.logspace(13, 14, 20)
info = recovery.plot_panel(c1, 'cross', zbins, mbins, transpose=True)

### 2D plots

In [ ]:
zbins = np.linspace(0, 2, 10)
mbins = np.logspace(13, 14, 5)

info = recovery.plot2D(c1, 'cross', zbins, mbins)
plt.show()
info = recovery.plot2D(c1, 'cross', zbins, mbins,
                       add_num=True, num_kwargs={'fontsize':15})

## Distances of matching
Here we evaluate the distance between the cluster centers and their redshifts. These distances can be shown for all matched clusters, or in bins:

In [ ]:
from clevar.match_metrics import distances

In [ ]:
info = distances.central_position(
    c1, c2, 'cross', radial_bins=20, radial_bin_units='degrees')

In [ ]:
info = distances.central_position(
    c1, c2, 'cross', radial_bins=20, radial_bin_units='degrees',                     
    quantity_bins='mass', bins=mbins, log_quantity=True)

In [ ]:
info = distances.central_position(
    c1, c2, 'cross', radial_bins=20, radial_bin_units='degrees',
    quantity_bins='z', bins=zbins[::2], log_quantity=False)

In [ ]:
info = distances.redshift(c1, c2, 'cross', redshift_bins=20, normalize='cat1')

In [ ]:
info = distances.redshift(
    c1, c2, 'cross', redshift_bins=20, normalize='cat1',
    quantity_bins='mass', bins=mbins, log_quantity=True)

In [ ]:
info = distances.redshift(
    c1, c2, 'cross', redshift_bins=20, normalize='cat1',
    quantity_bins='z', bins=zbins[::2], log_quantity=False)

The full information of the distances is outputed in a dictionary containing:

* `distances`: values of distances.
* `data`: Binned data used in the plot. It has the sections:

    * `hist`: Binned distances with (distance_bins, bin2). bins where no cluster was found have nan value.
    * `distance_bins`: The bin edges for distances.
    * `bins2` (optional): The bin edges along the second dimension.


In [ ]:
info.keys()

## Scaling Relations

In [ ]:
from clevar.match_metrics import scaling

### Redshift plots

#### Simple plot

In [ ]:
info = scaling.redshift(c1, c2, 'cross')

#### Color points by $\log(M)$ value

In [ ]:
info = scaling.redshift_masscolor(c1, c2, 'cross', add_err=True)

#### Color points by density at plot

In [ ]:
info = scaling.redshift_density(c1, c2, 'cross', add_err=True)

In [ ]:
info = scaling.redshift_density(c1, c2, 'cross', ax_rotation=45)

#### Split data into mass bins

In [ ]:
info = scaling.redshift_masspanel(c1, c2, 'cross', add_err=True)

#### Split data into mass bins and color by density

In [ ]:
info = scaling.redshift_density_masspanel(c1, c2, 'cross', add_err=True)

#### Evaluate metrics of the distribution

In [ ]:
info = scaling.redshift_metrics(c1, c2, 'cross')

In [ ]:
info = scaling.redshift_density_metrics(c1, c2, 'cross', ax_rotation=45)

In [ ]:
info = scaling.redshift_density_dist(c1, c2, 'cross', ax_rotation=45, add_err=False)

#### All of these functions with scatter plot can also fit a relation:

In [ ]:
info = scaling.redshift_density_metrics(
    c1, c2, 'cross', ax_rotation=45,
    add_fit=True, fit_bins1=20)

The full information of the scaling relation is outputed to a dictionay containing:

* `binned_data` (optional): input data for fitting, with values:

  * `x`: x values in fit (log of values if log=True).
  * `y`: y values in fit (log of values if log=True).
  * `y_err`: errorbar on y values (error_log if log=True).

* `fit` (optional): fitting output dictionary, with values:

    * `pars`: fitted parameter.
    * `cov`: covariance of fitted parameters.
    * `func`: fitting function with fitted parameter.
    * `func_plus`: fitting function with fitted parameter plus 1x scatter.
    * `func_minus`: fitting function with fitted parameter minus 1x scatter.
    * `func_scat`: scatter of fited function.
    * `func_dist`: `P(y|x)` - Probability of having y given a value for x, assumes normal distribution and uses scatter of the fitted function.
    * `func_scat_interp`: interpolated scatter from data.
    * `func_dist_interp`: `P(y|x)` using interpolated scatter.

* `plots` (optional): additional plots:

    * `fit`: fitted data
    * `errorbar`: binned data


In [ ]:
info['fit']['pars']

#### Evaluate the distribution
See how the distribution of mass happens in each bin for one of the catalogs

In [ ]:
%%time
info = scaling.redshift_dist_self(
    c2, redshift_bins_dist=21,
    mass_bins=[10**13.0, 10**13.2, 10**13.5, 1e15],
    redshift_bins=4, shape='line',
    fig_kwargs={'figsize':(15, 6)})

Compare with the distribution on the other catalog

In [ ]:
%%time
info = scaling.redshift_dist(
    c1, c2, 'cross', redshift_bins_dist=21,
    mass_bins=[10**13.0, 10**13.2, 10**13.5, 1e15],
    redshift_bins=4, shape='line',
    fig_kwargs={'figsize':(15, 6)})

### Mass plots

#### Simple plot

In [ ]:
info = scaling.mass(c1, c2, 'cross', add_err=True)

#### Color points by redshift value

In [ ]:
info = scaling.mass_zcolor(c1, c2, 'cross', add_err=True)

#### Color points by density at plot

In [ ]:
info = scaling.mass_density(c1, c2, 'cross', add_err=True)

#### Split data into redshift bins

In [ ]:
info = scaling.mass_zpanel(c1, c2, 'cross', add_err=True)

#### Split data into redshift bins and color by density

In [ ]:
info = scaling.mass_density_zpanel(c1, c2, 'cross', add_err=True)

#### Evaluate metrics of the distribution

In [ ]:
info = scaling.mass_metrics(c1, c2, 'cross')

In [ ]:
info = scaling.mass_density_metrics(c1, c2, 'cross', ax_rotation=45)

In [ ]:
info = scaling.mass_density_dist(c1, c2, 'cross', ax_rotation=45,
                                 add_err=False, plt_kwargs={'s':5})

#### All of these functions with scatter plot can also fit a relation:

In [ ]:
info = scaling.mass_density_metrics(
    c1, c2, 'cross', ax_rotation=45,
    add_fit=True, fit_bins1=8)

The full information of the scaling relation is outputed to a dictionay containing:

* `fit` (optional): fitting output dictionary, with values:

    * `pars`: fitted parameter.
    * `cov`: covariance of fitted parameters.
    * `func`: fitting function with fitted parameter.
    * `func_plus`: fitting function with fitted parameter plus 1x scatter.
    * `func_minus`: fitting function with fitted parameter minus 1x scatter.
    * `func_scat`: scatter of fited function.
    * `func_chi`: sqrt of chi_square(x, y) for the fitted function.

* `plots` (optional): additional plots:

    * `fit`: fitted data
    * `errorbar`: binned data


#### Evaluate the distribution
See how the distribution of mass happens in each bin for one of the catalogs

In [ ]:
%%time
info = scaling.mass_dist_self(
    c2, mass_bins_dist=21,
    mass_bins=[10**13.0, 10**13.2, 10**13.5, 1e14, 1e15],
    redshift_bins=4, shape='line',
    fig_kwargs={'figsize':(15, 6)})

Compare with the distribution on the other catalog

In [ ]:
%%time
info = scaling.mass_dist(
    c1, c2, 'cross', mass_bins_dist=21,
    mass_bins=[10**13.0, 10**13.2, 10**13.5, 1e14, 1e15],
    redshift_bins=4, shape='line',
    fig_kwargs={'figsize':(15, 6)})